Dans ce notebook, on essaye de mettre en place une méthode d'analogie qui exhibe les stéréotypes de genre présentée dans l'article écrit par Bolukbasi. 

Dans les tâches d'analogie standard, on nous donne trois mots, par exemple "he", "she", "king", et on cherche le 4ᵉ mot pour résoudre la relation "he" vers "king" est comme "she" vers "x". Ici, nous modifions la tâche d'analogie de manière à ce qu'avec deux mots donnés, par exemple "he" et "she", nous voulons générer une paire de mots, "x" et "y", telle que la relation "he" vers "x" soit similaire à la relation "she" vers "y" et constitue une bonne analogie. 

Cette modification nous permet de générer systématiquement des paires de mots que l'embedding considère comme analogues à "he" et "she" (ou à toute autre paire de mots de référence).

L'entrée dans notre générateur d'analogies est une paire de mots de départ (a, b) déterminant une direction de départ vect(a) − vect(b) correspondant à la différence normalisée entre les deux mots de départ. Dans la tâche ci-dessous, nous utilisons (a, b) = (she, he). Nous évaluons ensuite toutes les paires de mots x, y en utilisant la métrique suivante :

$S(a,b)(x, y) = \begin{cases} 
\cos(\mathbf{\tilde{a} - \tilde{b}}, \mathbf{\tilde{x} - \tilde{y}}) & \text{if } \|\mathbf{\tilde{x} - \tilde{y}}\| \leq \delta \\
0 & \text{otherwise}
\end{cases}$


où δ est un seuil de similarité. 

L'intuition de la métrique de notation est que nous souhaitons qu'une paire d'analogies soit proche et parallèle à la direction de départ, tandis que les deux mots ne doivent pas être trop éloignés pour être sémantiquement cohérents. 

Le paramètre δ établit le seuil de similarité sémantique. En pratique, il est choisi égal à 1 dans l'article mais **à étudier**. 

C'est parti !!

In [2]:
from gensim import models
import numpy as np

In [3]:
first_model = models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin\GoogleNews-vectors-negative300.bin', binary=True)

In [4]:
first_model.most_similar('man')

[('woman', 0.7664012908935547),
 ('boy', 0.6824871301651001),
 ('teenager', 0.6586930155754089),
 ('teenage_girl', 0.6147903203964233),
 ('girl', 0.5921714305877686),
 ('suspected_purse_snatcher', 0.571636438369751),
 ('robber', 0.5585119128227234),
 ('Robbery_suspect', 0.5584409832954407),
 ('teen_ager', 0.5549196600914001),
 ('men', 0.5489763021469116)]

**Etape 1 :** traitement des données. \
On traite le word embedding selon la même procédure que celle présentée dans l'article. 

In [5]:
first_model.unit_normalize_all()

In [6]:
first_model.sort_by_descending_frequency()

In [7]:
first_model.index_to_key[:10]

['</s>', 'in', 'for', 'that', 'is', 'on', '##', 'The', 'with', 'said']

In [8]:
def verifier_chaine(chaine) :
    if chaine.islower() and chaine.isalpha() and len(chaine)<20 :
        return True
    return False

In [9]:
words_sorted = first_model.index_to_key[:50000]
words_sorted = [word for word in words_sorted if verifier_chaine(word)]
words_sorted[:10], len(words_sorted)

(['in', 'for', 'that', 'is', 'on', 'with', 'said', 'was', 'the', 'at'], 24065)

In [10]:
vector_size = first_model.vector_size
model = models.keyedvectors.KeyedVectors(vector_size, count=0)

In [11]:
for word in words_sorted :
    model[word] = first_model[word]

In [12]:
model.most_similar('man')

[('woman', 0.7664012908935547),
 ('boy', 0.6824870109558105),
 ('teenager', 0.6586930155754089),
 ('girl', 0.5921714305877686),
 ('robber', 0.5585119128227234),
 ('men', 0.5489763021469116),
 ('guy', 0.5420035719871521),
 ('person', 0.5342026352882385),
 ('gentleman', 0.5337990522384644),
 ('motorcyclist', 0.5336882472038269)]

On a fini la première étape de traitement des données. Question reste en suspens : pourquoi je n'ai pas le même nombre de mots dans mon word embedding filtré que dans l'article ?

Maintenant, on passe à l'étape 2. Codons l'algorithme d'analogies de l'article ! 

In [13]:
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

Les vecteurs de mot ont été normalisés, donc calculer la cosinus-similarité revient à calculer le produit scalaire entre deux vecteurs. Ce produit scalaire peut être calculé par la fonction linear kernel : \
La fonction linear_kernel prend en entrée deux matrices X et Y, où chaque ligne de chaque matrice représente un échantillon (donc dans notre cas, un mot) et chaque colonne représente une caractéristique (une des coordonnées du vecteur associé). Elle renvoie une matrice où l'élément à la position (i, j) est le noyau linéaire entre le i-ème échantillon de la première matrice (X) et le j-ème échantillon de la deuxième matrice (Y). Donc dans notre cas, elle renvoie la similarité entre le mot i de la matrice X et le mot j de la matrice Y. 

In [16]:
test1 = np.ones((3,4))
print(linear_kernel(test1,test1))
#on fait le linear kernel entre deux matrices qui ont 3 mots chacune
#donc on calcule la similarité entre 9 mots différents
#la matrice contient effectivement 9 valeurs
print(cosine_similarity(test1,test1))
#linear kernel et cosine similarity ne sont pas toujours la même chose
#si la matrice n'est pas normalisée, les valeurs ne sont pas les mêmes

[[4. 4. 4.]
 [4. 4. 4.]
 [4. 4. 4.]]
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]


In [14]:
#on veut une matrice X qui contient les vecteurs de chaque mot
#elle a donc nb_words lignes et vector_size colonnes
X = model.vectors
X.shape

(24065, 300)

Si on fait linear_kernel(X,X), la matrice va nous donner la cosinus_similarité de toutes les paires de mots possibles. Ce sera une matrice symétrique. 

In [15]:
cos_sim = linear_kernel(X)
#quand on ne précise pas de deuxième matrice, la fonction calcule la
#similarité entre chaque ligne de X et elle-même, donc linear_kernel(X,X)
print(cos_sim.shape)
cos_sim[1,2] == cos_sim[2,1]

(24065, 24065)


True

Maintenant qu'on a compris comment fonctionne linear_kernel, appliquons le dans notre algoroithme d'analogies. 

In [17]:
#on trouve les vecteurs associés à she et he 
vect_she = model.get_vector('she')
vect_he = model.get_vector('he')

Comme nos vecteurs sont normalisées, on a pour x,y des vecteurs quelconques du word embedding et a,b des vecteurs fixes (qui ici joueront le rôle de he,she) : 

$$ 
\cos(\mathbf{a}-\mathbf{b}, \mathbf{x}-\mathbf{y}) = (\mathbf{a}-\mathbf{b})^T \cdot (\mathbf{x}-\mathbf{y}) = \mathbf{a}^T \cdot \mathbf{x} - \mathbf{b}^T \cdot \mathbf{x} - \mathbf{a}^T \cdot \mathbf{y} + \mathbf{b}^T \cdot \mathbf{y}
$$


Je veux calculer une matrice de similarité mat_sim telle que, en notant i,j les index associés respectivement aux mots x et y, on a : sim_mat(i,j) = cos(a-b,x-y).

In [18]:
diff_visée = (vect_she - vect_he)/np.linalg.norm(vect_she - vect_he)

In [19]:
mat_1 = linear_kernel((diff_visée).reshape(1,-1), X) 
#me donne une ligne et 24065 colonnes
#chaque colonne correspond à la cos_sim entre un mot et le vect diff she-he
#attention, il faut normaliser la diff pour pouvoir utilsier linear_kernel

In [20]:
nb_words = len(model.index_to_key)

In [21]:
sim_mat = np.zeros((nb_words,nb_words))

xmat_1 = np.array(mat_1[0,:])

sim_mat = np.tile(xmat_1[np.newaxis,:], (nb_words,1))-np.tile(xmat_1[:,np.newaxis], (1,nb_words))

In [22]:
sim_mat.shape

(24065, 24065)

In [23]:
dir = vect_she - vect_he
n = 9
p = 200
print(model.index_to_key[n])
print(model.index_to_key[p])
diff = model[n] - model[p]
sim = cosine_similarity(dir.reshape(1, -1), diff.reshape(1, -1))[0,0]
print(sim)
print(sim_mat[n,p])
print(sim_mat[n,p]-sim)

at
months
-0.037064265
0.04987895
0.08694322


On observe une légère différence dans les deux calculs de la cos_similarité mais leur différence semble toujours du même signe. On peut espérer que cela n'influe pas trop sur l'ordre des valeurs de coefficient. 

Vérifions si les résultats trouvés sont les mêmes que ceux de l'article. 

Maintenant qu'on a une matrice qui nous donne les similarités, on va chercher les paires de mots associées aux plus gros coefficients, et qui vérifient que norm(x-y) < 1 (cf. condition dans l'article). 

Méthode initiale : 

In [ ]:
# best_coefs = -np.ones(10)
# indices_best = [0]*10
# for i in range(nb_words):
#     if i%1000 == 0 :
#         print(i, 'mots traités sur', nb_words)
#     for j in range(i+1, nb_words): 
#         if sim_mat[i,j] > np.min(best_coefs):
#             if np.linalg.norm(model[i]-model[j]) < 1 : 
#                 index = np.argmin(best_coefs)
#                 best_coefs[index] = sim_mat[i,j] 
#                 indices_best[index] = (i,j)

Plus efficace : faire un tri des coefs puis parmi les coefs, regardez ceux qui vérifient la condition sur la norme. Pour le tri des coefs, on peut faire un tri partiel : commencer par trier les 10 000 premiers coefs et voir si parmi eux, on peut pas en avoir 10 qui vérifient la condition de norme. Regarder partial sort dans la doc --> fonction argpartition : je commence à trier les 1000 premiers, je fais mon test et si j'ai déjà les 10, je m'arrête. Algo itératif. Il faut mettre la matrice sim_mat à plat, regarder les arguments donnés par argpartition, utiliser unravel pour retrouver le i,j qui correspond à cet indice. 


In [33]:
# Supposons que sim_mat soit votre matrice de similarité et model votre array de vecteurs
best_coefs = -np.ones(100)
indices_best = [0] * 100

# Utilisez argpartition pour obtenir les indices triés des 1000 plus grands coefficients de toute la matrice
sorted_indices = np.argpartition(-sim_mat, 1000000, axis=None)[:1000000]

# Utilisez unravel_index pour obtenir les indices dans la matrice d'origine
indices_in_sim_mat = np.unravel_index(sorted_indices, sim_mat.shape)

# Parcourez les indices et mettez à jour les listes best_coefs et indices_best
for i, j in zip(*indices_in_sim_mat):
    if sim_mat[i, j] > np.min(best_coefs):
        if np.linalg.norm(model[i] - model[j]) < 1:
            index = np.argmin(best_coefs)
            best_coefs[index] = sim_mat[i, j]
            indices_best[index] = (i, j)

In [34]:
print(best_coefs)
print(indices_best)

[0.37701178 0.61596274 0.48028737 0.37364566 0.4798581  0.51472658
 0.48976302 0.3735052  0.39002678 0.37368891 0.41738576 0.56172156
 0.46421447 0.48796439 0.38053083 0.44171065 0.42500311 0.44685441
 0.41985148 0.44541574 0.45323342 0.40022433 0.44366646 0.37632149
 0.40132248 0.39969683 0.40044248 0.39130366 0.44063488 0.39350483
 0.3793433  0.41783339 0.38414389 0.4213554  0.47627163 0.44463554
 0.44506505 0.39175555 0.46092471 0.42772996 0.46319872 0.40993339
 0.38716882 0.39778835 0.40692717 0.37255162 0.37229171 0.48022708
 0.38284594 0.38280618 0.45015118 0.40670905 0.45971811 0.45190045
 0.37843674 0.38927019 0.42116946 0.37548172 0.38947111 0.41760045
 0.47507542 0.37999514 0.38019988 0.37497556 0.44278529 0.40464377
 0.37855807 0.3820484  0.73908675 0.54357237 0.45801437 0.39602003
 0.39527562 0.87977844 0.38095343 0.37348789 0.3823114  0.37600511
 0.38411486 0.37668198 0.54205835 0.37759784 0.42082185 0.37737972
 0.43038878 0.42257112 0.39799502 0.46753994 0.43175265 0.4374

In [35]:
for i,j in indices_best :
    analog_she = model.index_to_key[i]
    analog_he = model.index_to_key[j]
    print(analog_she)
    print(analog_he)
    print('')

uncle
aunts

hero
heroine

gentleman
lady

surgeon
nurse

fatherhood
motherhood

man
woman

man
girl

beard
ponytail

beard
wig

fathers
mothers

fraternity
sorority

businessman
businesswoman

king
princess

king
queen

youngsters
girls

myself
herself

actor
actresses

fella
gal

actor
actress

nephew
mother

nephew
daughter

nephew
husband

nephew
sister

nephew
daughters

father
daughter

father
mom

nephew
granddaughter

nephew
niece

kid
mommy

father
mother

nephew
stepdaughter

kid
mom

nephew
granddaughters

kid
girl

lad
schoolgirl

football
softball

football
volleyball

father
sister

patriarch
matriarch

shirt
blouse

dude
gal

dude
chick

dad
mommy

brother
niece

brother
granddaughter

brother
aunt

brother
grandmother

brother
sisters

brother
boyfriend

brother
daughters

brother
sister

brother
husband

brother
daughter

brother
mother

member
chairwoman

protagonist
heroine

tall
petite

barber
salon

uncle
niece

kings
queens

kings
queen

councilman
councilwoman

b